<a href="https://colab.research.google.com/github/MarinaZRocha/Mod_Ling_Robotica/blob/main/E02_Mod_ling_179741.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Universidade Federal do Rio Grande (FURG)
### Programa de Pós-Graduação em Computação (PPGComp)
#### Disciplina: ModLing para Robótica 2025
##### Prof. Rodrigo da Silva Guerra
##### E02: Sistema de Recomendação usando Word2Vec

### Descrição do Exercício

Neste exercício, o objetivo é construir um sistema de recomendação de produtos com base no histórico de compras de um e-commerce. A premissa é que a sequência de produtos adquiridos por um cliente em uma única transação possui uma relação semântica. Por exemplo, um cliente que compra itens para uma festa de aniversário provavelmente selecionará produtos relacionados a esse tema. Utilizaremos o algoritmo Word2Vec para capturar essas relações, tratando cada produto como uma "palavra" e cada compra (carrinho de compras) como uma "frase".

Você baixará a base de dados e deverá realizar todo o processo de tratamento, treinamento do modelo e implementação das funcionalidades de recomendação.

### Instruções

O exercício é demonstrado [nessa videoaula](https://youtu.be/PKOGlgVhS_A), mas você é encorajado a tentar implementar por conta própria.

1.  **Carregamento e Pré-processamento dos Dados:**
    * Faça o download do arquivo [`retail.xlsx`](https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n).
    * Carregue os dados em um DataFrame do Pandas.
    * Realize a limpeza dos dados, removendo registros com valores nulos.
    * Converta a coluna de códigos de produtos (`StockCode`) para o formato de string.

2.  **Criação do Dicionário de Produtos:**
    * Crie um dicionário Python que mapeie os códigos dos produtos (`StockCode`) às suas respectivas descrições (`Description`). Este dicionário será utilizado para a interpretação dos resultados, permitindo a visualização das descrições dos produtos recomendados.

3.  **Preparação dos Dados para Treinamento:**
    * O histórico de compras de cada cliente deve ser tratado como uma "frase", onde cada produto é uma "palavra". Agrupe os produtos por cada `InvoiceNo`, criando uma lista de listas, onde cada lista interna representa uma compra.

4.  **Treinamento do Modelo Word2Vec:**
    * Utilize a biblioteca `gensim` para treinar um modelo Word2Vec com as "frases" (compras) criadas no passo anterior.

5.  **Desenvolvimento das Funções de Recomendação:**
    * **Função 1: Produtos Similares**
        * Implemente uma função que, ao receber um código de produto, retorne uma lista dos produtos mais similares, com base no espaço latente aprendido pelo Word2Vec. A função deve exibir a descrição do produto de entrada e as descrições dos produtos mais similares, juntamente com seus respectivos scores de similaridade.
    * **Função 2: Recomendação por Carrinho de Compras**
        * Implemente uma função que receba uma lista de códigos de produtos (simulando um carrinho de compras).
        * Calcule o vetor médio de embedding para os produtos presentes no carrinho.
        * Utilize o vetor médio para encontrar os produtos mais próximos no espaço de embedding, que serão as recomendações.
        * A função deve retornar uma lista com as descrições dos produtos recomendados, excluindo aqueles que já estão no carrinho, e seus respectivos scores de similaridade.

### Código Inicial

In [1]:
# Código para baixar os dados
!gdown https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n

Downloading...
From: https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n
To: /content/retail.xlsx
100% 23.7M/23.7M [00:00<00:00, 43.0MB/s]


In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_excel('retail.xlsx')


In [4]:
df.dropna(inplace=True)

In [5]:
product_dict = dict(zip(df['StockCode'], df['Description']))

In [6]:
product_dict['10123C']

'HEARTS WRAPPING TAPE '

In [7]:
customers = df['CustomerID'].unique().tolist()
len(customers)

4372

In [25]:
def client_history(customer_id):

  purchases = []
  tt_history = df.groupby(["CustomerID", "InvoiceNo"])['StockCode'].apply(list).reset_index()
  for client in customer_id:
    purchase = tt_history.groupby(['CustomerID'])['StockCode'].apply(list)
    purchases.append(purchase)

  #purchases = tt_history.groupby(['CustomerID'])['StockCode'].apply(list).tolist()
  return purchases


In [26]:
client_history(customers)

[CustomerID
 12346.0                                   [[23166], [23166]]
 12347.0    [[85116, 22375, 71477, 22492, 22771, 22772, 22...
 12348.0    [[84992, 22951, 84991, 84991, 21213, 21213, 22...
 12349.0    [[23112, 23460, 21564, 21411, 21563, 22131, 22...
 12350.0    [[21908, 22412, 79066K, 79191C, 22348, 84086C,...
                                  ...                        
 18280.0    [[82484, 22180, 22467, 22725, 22727, 22495, 22...
 18281.0    [[22037, 22716, 22028, 23007, 23008, 23209, 22...
 18282.0    [[21270, 23187, 23295, 22089, 21108, 21109, 22...
 18283.0    [[22356, 20726, 22384, 22386, 20717, 20718, 85...
 18287.0    [[22755, 22754, 22753, 22756, 22758, 22757, 22...
 Name: StockCode, Length: 4372, dtype: object,
 CustomerID
 12346.0                                   [[23166], [23166]]
 12347.0    [[85116, 22375, 71477, 22492, 22771, 22772, 22...
 12348.0    [[84992, 22951, 84991, 84991, 21213, 21213, 22...
 12349.0    [[23112, 23460, 21564, 21411, 21563, 22131, 22...

In [ ]:
train_size = (0.9*len(customers)).astype(int)

cl_train = customers[:train_size]
cl_test = customers[train_size:len(customers)]

df_train = df[df['CustomerID'].isin(cl_train)]
df_test = df[df['CustomerID'].isin(cl_test)]